<a href="https://colab.research.google.com/github/alyrazik/News_content_collect_store/blob/MongoDB-Saving-and-querying/CrawlingBBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  News Content Collect and Store
## by Aly Abdelrazek
 
 


##Setup and Importing libraries

In [1]:
!python -m pip install pymongo[srv]

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
import pymongo
from pymongo import MongoClient
from datetime import datetime


In [3]:
pymongo.version

'3.11.0'

##Global variables

In [30]:
news_page_url = "https://bbc.com/news"
parent = "https://bbc.com"
SEARCH_LIMIT = 10   #limit the number of returned articles matching a keyword search


##Crawl


In [17]:
# Obtains news stories links from news_page_url (https://bbc.com/news)
def news_page_to_news_urls(url, parent):
    ''' Takes the URL to a news page and the prefix of the absolute link, and returns a list of news stories URLs.'''
    news_urls = []
    news_page_html = requests.get(url).text
    news_page_soup = BeautifulSoup(news_page_html, "html.parser")
    news_page_links = news_page_soup.findAll("a") #find all links
    
    for link in news_page_links:
        href = link.get("href")
        if href.startswith("/news") and href[-1].isdigit(): #news articles start with /news and ends with a digit
            news_url = parent + href
            s = BeautifulSoup(requests.get(news_url).text , "html.parser")
            if s.find(class_= "css-16rg7hm-ContainerWithSidebarWrapper e1jl38b40") is not None: #this class is only available in news articles
              news_urls.append(news_url)
    return news_urls

news_urls = news_page_to_news_urls(news_page_url, absolute_link_prefix)

In [18]:
len(news_urls)

38

## Scrape

In [ ]:
documents = []
document_index = 0

for url in news_urls:
    print("Fetching {}".format(url))
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    news_heading = (soup.find('h1')).text
    news_text = [p.text for p in soup.find('article').find_all('p')]
    news_time = (soup.find('time'))['datetime']

    documents.append({"Document_Index":str(document_index), "Heading":news_heading, "Article":news_text, "DateTime":news_time})
    document_index = document_index+1


##Saving to MongoDB

In [47]:
#connect to client
client = MongoClient("mongodb+srv://aly:a@cluster0.4pfcp.mongodb.net/db?retryWrites=true&w=majority")

In [48]:
#create a database
db = client["news_database"]

In [49]:
#create a collection (a table)
bbc_news = db["bbc_news"]

In [65]:
db['bbc_news'].delete_many({})

In [66]:
bbc_news.insert_many(documents)

##Retrieving content from MongoDB

In [67]:
def retrieve_documents(database, collection):
  '''takes a MongoDB database and collection name, and returns all documents in collection to a pandas dataframe'''
  retrieved_documents = database[collection].find() #do not use database.collection, it is a string :)
  retrieved_df = pd.DataFrame(retrieved_documents)
  return retrieved_df

db = client["news_database"]
df = retrieve_documents(database = db , collection = 'bbc_news')

## Keyword search in article text

In [68]:
db = client["news_database"]
db.bbc_news.ensure_index([
          ("Article", "text"),
          ("Heading", "text"),
  ],
  name="search_index",
  weights={
      'Article':25,
      'Heading':100
  }
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: ensure_index is deprecated. Use create_index instead.
  if __name__ == '__main__':


In [69]:
def search(search_text):
    returned_cursor = bbc_news.find({"$text": {"$search": search_text}}).limit(SEARCH_LIMIT)
    df = pd.DataFrame(returned_cursor)
    return df


In [70]:
output = search("france")

In [71]:
output

,_id,Document_Index,Heading,Article,DateTime
0,5f9b3fa64f790f3d240e148c,28,France attack: Three killed in 'Islamist terro...,[.css-14iz86j-BoldText{font-weight:bold;}Three...,2020-10-29T21:34:43.000Z
1,5f9b3fa64f790f3d240e1473,3,France attack: Three killed in 'Islamist terro...,[.css-14iz86j-BoldText{font-weight:bold;}Three...,2020-10-29T21:34:43.000Z
2,5f9b3fa64f790f3d240e147c,12,"Covid: Merkel warns of 'long, hard winter' as ...",[.css-14iz86j-BoldText{font-weight:bold;}Germa...,2020-10-29T19:12:05.000Z
3,5f9b3fa64f790f3d240e147a,10,"Covid: Merkel warns of 'long, hard winter' as ...",[.css-14iz86j-BoldText{font-weight:bold;}Germa...,2020-10-29T19:12:05.000Z
4,5f9b3fa64f790f3d240e1479,9,US growth surged in summer but recovery is slo...,[.css-14iz86j-BoldText{font-weight:bold;}The U...,2020-10-29T14:28:58.000Z
